# Unified Risk Model Pipeline - Complete Test\n\nBu notebook tek bir pipeline ile tüm risk modelleme işlemlerini göstermektedir:\n\n1. Otomatik veri oluşturma\n2. WOE transformasyonu ve univariate Gini\n3. PSI -> VIF -> Correlation -> IV -> Boruta -> Stepwise selection\n4. Tüm ML modelleri (Logistic, GAM, CatBoost, LightGBM, XGBoost, RF, ExtraTrees)\n5. Stage 1 ve Stage 2 kalibrasyon\n6. Risk band optimizasyonu (Herfindahl, Hosmer-Lemeshow, Binomial testler)\n7. Kapsamlı raporlama

In [1]:
# Gerekli kütüphaneleri yükle
import sys
import os
import warnings
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# Parent directory'yi path'e ekle
sys.path.insert(0, os.path.abspath('../'))

# Pipeline bileşenlerini import et
from src.risk_pipeline.core.config import Config
from src.risk_pipeline.unified_pipeline import UnifiedRiskPipeline

print("Kütüphaneler başarıyla yüklendi!")
print(f"Çalışma dizini: {os.getcwd()}")
print(f"Python versiyonu: {sys.version}")

Kütüphaneler başarıyla yüklendi!
Çalışma dizini: C:\Users\Acer\risk-model-pipeline-dev\notebooks
Python versiyonu: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]


## 1. Sentetik Veri Oluşturma

In [2]:
# Rastgelelik için seed ayarla
np.random.seed(42)

# Sentetik kredi riski verisi oluştur
n_samples = 15000

# Temel özellikler
data = pd.DataFrame({
    'customer_id': range(n_samples),
    'application_date': pd.date_range(
        start='2020-01-01', 
        periods=n_samples, 
        freq='6H'
    ),
    
    # Numerik değişkenler
    'age': np.random.normal(40, 12, n_samples).clip(18, 80).astype(int),
    'income': np.random.lognormal(10.5, 0.6, n_samples),
    'loan_amount': np.random.lognormal(9.5, 0.8, n_samples),
    'employment_years': np.random.exponential(5, n_samples).clip(0, 40),
    'credit_score': np.random.normal(650, 100, n_samples).clip(300, 850),
    'debt_to_income': np.random.beta(2, 5, n_samples) * 100,
    'num_credit_lines': np.random.poisson(3, n_samples),
    'months_since_delinquent': np.random.exponential(24, n_samples),
    'total_credit_limit': np.random.lognormal(10, 0.7, n_samples),
    'utilization_rate': np.random.beta(3, 5, n_samples) * 100,
    
    # Kategorik değişkenler
    'home_ownership': np.random.choice(['RENT', 'OWN', 'MORTGAGE', 'OTHER'], 
                                      n_samples, p=[0.35, 0.20, 0.40, 0.05]),
    'loan_purpose': np.random.choice(['debt_consolidation', 'credit_card', 
                                     'home_improvement', 'major_purchase', 'other'], 
                                    n_samples, p=[0.35, 0.20, 0.15, 0.15, 0.15]),
    'employment_type': np.random.choice(['Full-time', 'Part-time', 'Self-employed', 
                                        'Unemployed', 'Retired'], 
                                       n_samples, p=[0.60, 0.15, 0.15, 0.05, 0.05]),
    'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD', 'Other'], 
                                 n_samples, p=[0.30, 0.40, 0.20, 0.05, 0.05]),
    'marital_status': np.random.choice(['Single', 'Married', 'Divorced', 'Widowed'], 
                                      n_samples, p=[0.30, 0.50, 0.15, 0.05]),
    'state': np.random.choice(['CA', 'NY', 'TX', 'FL', 'PA', 'IL', 'OH', 'GA', 'NC', 'MI'],
                             n_samples)
})

# Eksik değerler ekle (gerçekçi olması için)
missing_cols = ['months_since_delinquent', 'employment_years', 'utilization_rate']
for col in missing_cols:
    missing_idx = np.random.choice(n_samples, size=int(0.1 * n_samples), replace=False)
    data.loc[missing_idx, col] = np.nan

# Hedef değişken oluştur (gerçekçi default pattern'leri ile)
default_prob_base = 0.08  # Temel default oranı

# Risk skoru hesapla
risk_score = (
    (data['credit_score'] < 600).astype(float) * 0.25 +
    (data['debt_to_income'] > 40).astype(float) * 0.20 +
    (data['loan_amount'] / data['income'] > 0.5).astype(float) * 0.15 +
    (data['employment_years'] < 2).astype(float) * 0.10 +
    (data['home_ownership'] == 'RENT').astype(float) * 0.10 +
    (data['utilization_rate'] > 80).astype(float) * 0.10 +
    (data['employment_type'] == 'Unemployed').astype(float) * 0.10
)

# Gürültü ekle
risk_score += np.random.normal(0, 0.05, n_samples)

# Default olasılığını hesapla
default_prob_adjusted = default_prob_base * (1 + risk_score * 2.5)
default_prob_adjusted = np.clip(default_prob_adjusted, 0, 0.5)
data['default'] = (np.random.random(n_samples) < default_prob_adjusted).astype(int)

print(f"Veri seti oluşturuldu: {n_samples} kayıt")
print(f"Default oranı: {data['default'].mean():.2%}")
print(f"Boyut: {data.shape}")
print(f"\nÖzellikler: {list(data.columns)}")
print(f"\nİlk 5 satır:")
data.head()

Veri seti oluşturuldu: 15000 kayıt
Default oranı: 12.81%
Boyut: (15000, 19)

Özellikler: ['customer_id', 'application_date', 'age', 'income', 'loan_amount', 'employment_years', 'credit_score', 'debt_to_income', 'num_credit_lines', 'months_since_delinquent', 'total_credit_limit', 'utilization_rate', 'home_ownership', 'loan_purpose', 'employment_type', 'education', 'marital_status', 'state', 'default']

İlk 5 satır:


,customer_id,application_date,age,income,loan_amount,employment_years,credit_score,debt_to_income,num_credit_lines,months_since_delinquent,total_credit_limit,utilization_rate,home_ownership,loan_purpose,employment_type,education,marital_status,state,default
0,0,2020-01-01 00:00:00,45,33321.098707,2739.532121,1.822872,510.601996,18.042761,3,NaN,6583.662106,36.216158,RENT,credit_card,Part-time,Bachelor,Single,CA,1
1,1,2020-01-01 06:00:00,38,35610.878918,5744.576564,7.168679,568.446315,9.224136,2,NaN,22535.996273,14.791731,RENT,other,Self-employed,High School,Married,NC,0
2,2,2020-01-01 12:00:00,47,37743.816018,8353.010359,3.291365,847.762386,44.379225,2,1.029136,24181.888050,37.934405,MORTGAGE,debt_consolidation,Full-time,Bachelor,Divorced,IL,1
3,3,2020-01-01 18:00:00,58,64094.694924,15059.060698,9.425473,533.344587,21.626382,5,14.771954,8890.186718,26.375190,MORTGAGE,major_purchase,Full-time,Bachelor,Divorced,OH,0
4,4,2020-01-02 00:00:00,37,23194.480385,30312.936916,14.180700,620.178434,46.276800,1,74.013684,38768.961266,NaN,OTHER,debt_consolidation,Part-time,High School,Single,GA,0


## 2. Data Dictionary Oluşturma

In [3]:
# Veri sözlüğü oluştur
data_dictionary = pd.DataFrame([
    {'variable': 'age', 'description': 'Müşteri yaşı (yıl)', 'type': 'numeric', 'category': 'demographic'},
    {'variable': 'income', 'description': 'Yıllık gelir (USD)', 'type': 'numeric', 'category': 'financial'},
    {'variable': 'loan_amount', 'description': 'Talep edilen kredi tutarı', 'type': 'numeric', 'category': 'loan'},
    {'variable': 'employment_years', 'description': 'Mevcut işyerinde çalışma süresi', 'type': 'numeric', 'category': 'employment'},
    {'variable': 'credit_score', 'description': 'Kredi skoru', 'type': 'numeric', 'category': 'credit'},
    {'variable': 'debt_to_income', 'description': 'Borç/Gelir oranı (%)', 'type': 'numeric', 'category': 'financial'},
    {'variable': 'num_credit_lines', 'description': 'Açık kredi hesap sayısı', 'type': 'numeric', 'category': 'credit'},
    {'variable': 'months_since_delinquent', 'description': 'Son gecikme sonrası geçen ay', 'type': 'numeric', 'category': 'credit'},
    {'variable': 'total_credit_limit', 'description': 'Toplam kredi limiti', 'type': 'numeric', 'category': 'credit'},
    {'variable': 'utilization_rate', 'description': 'Kredi kullanım oranı (%)', 'type': 'numeric', 'category': 'credit'},
    {'variable': 'home_ownership', 'description': 'Ev sahipliği durumu', 'type': 'categorical', 'category': 'demographic'},
    {'variable': 'loan_purpose', 'description': 'Kredi amacı', 'type': 'categorical', 'category': 'loan'},
    {'variable': 'employment_type', 'description': 'İstihdam türü', 'type': 'categorical', 'category': 'employment'},
    {'variable': 'education', 'description': 'Eğitim seviyesi', 'type': 'categorical', 'category': 'demographic'},
    {'variable': 'marital_status', 'description': 'Medeni durum', 'type': 'categorical', 'category': 'demographic'},
    {'variable': 'state', 'description': 'Eyalet', 'type': 'categorical', 'category': 'geographic'}
])

print("Veri sözlüğü oluşturuldu:")
print(data_dictionary.head(10))

Veri sözlüğü oluşturuldu:
                  variable                      description     type  \
0                      age               Müşteri yaşı (yıl)  numeric   
1                   income               Yıllık gelir (USD)  numeric   
2              loan_amount        Talep edilen kredi tutarı  numeric   
3         employment_years  Mevcut işyerinde çalışma süresi  numeric   
4             credit_score                      Kredi skoru  numeric   
5           debt_to_income             Borç/Gelir oranı (%)  numeric   
6         num_credit_lines          Açık kredi hesap sayısı  numeric   
7  months_since_delinquent     Son gecikme sonrası geçen ay  numeric   
8       total_credit_limit              Toplam kredi limiti  numeric   
9         utilization_rate         Kredi kullanım oranı (%)  numeric   

      category  
0  demographic  
1    financial  
2         loan  
3   employment  
4       credit  
5    financial  
6       credit  
7       credit  
8       credit  
9       cre

## 3. Kalibrasyon Verileri Hazırlama

In [4]:
# Stage 1 kalibrasyon verisi (uzun dönem)
calibration_data = data.copy()
print(f"Stage 1 kalibrasyon verisi (uzun dönem):")
print(f"  Kayıt sayısı: {len(calibration_data)}")
print(f"  Default oranı: {calibration_data['default'].mean():.2%}")

# Stage 2 kalibrasyon verisi (son 3 ay)
recent_cutoff = data['application_date'].max() - pd.DateOffset(months=3)
stage2_data = data[data['application_date'] >= recent_cutoff].copy()

# Son dönemde default oranında hafif artış simüle et
additional_defaults = np.random.choice(stage2_data[stage2_data['default'] == 0].index, 
                                       size=int(len(stage2_data) * 0.02), replace=False)
stage2_data.loc[additional_defaults, 'default'] = 1

print(f"\nStage 2 kalibrasyon verisi (son dönem):")
print(f"  Kayıt sayısı: {len(stage2_data)}")
print(f"  Tarih aralığı: {stage2_data['application_date'].min()} - {stage2_data['application_date'].max()}")
print(f"  Default oranı: {stage2_data['default'].mean():.2%}")

Stage 1 kalibrasyon verisi (uzun dönem):
  Kayıt sayısı: 15000
  Default oranı: 12.81%

Stage 2 kalibrasyon verisi (son dönem):
  Kayıt sayısı: 361
  Tarih aralığı: 2030-01-07 18:00:00 - 2030-04-07 18:00:00
  Default oranı: 14.68%


## 4. Pipeline Konfigürasyonu

In [5]:
# Kapsamlı konfigürasyon oluştur
config = Config(
    # Temel ayarlar
    target_col='default',
    id_col='customer_id',
    time_col='application_date',
    random_state=42,
    
    # Pipeline modu
    enable_scoring=False,  # Skorlama varsayılan olarak kapalı
    enable_woe=True,       # WOE transformasyonu aktif
    enable_noise_sentinel=True,  # Gürültü değişkeni kontrolü
    
    # Veri bölme ayarları
    test_ratio=0.2,
    oot_months=6,
    equal_default_splits=True,  # Eşit default oranları
    min_oot_size=500,
    
    # Değişken seçim ayarları
    selection_order=['psi', 'vif', 'correlation', 'iv', 'boruta', 'stepwise'],
    selection_method='stepwise',  # forward/backward/stepwise
    max_features=20,
    
    # Binning ayarları
    binning_method='optimized',  # IV/Gini optimize binning
    min_bin_size=0.05,
    max_bins=10,
    monotonic_woe=True,
    
    # Eşik değerler
    psi_threshold=0.25,
    vif_threshold=10,
    correlation_threshold=0.9,
    iv_threshold=0.02,
    
    # İmputation ayarları
    numeric_imputation='median',
    categorical_imputation='missing',
    outlier_method='clip',
    min_category_freq=0.01,
    
    # Model ayarları
    model_type='all',  # Tüm modelleri eğit
    use_optuna=False,  # Hız için kapalı (açılabilir)
    n_optuna_trials=30,
    
    # Kalibrasyon ayarları
    enable_calibration=True,
    calibration_method='isotonic',  # isotonic/sigmoid/platt/beta
    stage2_method='lower_mean',     # lower_mean/upper_bound/weighted/shift
    
    # Risk band ayarları
    n_risk_bands=10,
    band_method='optimal',  # quantile/equal_width/optimal
    
    # Çıktı ayarları
    output_dir='../outputs',
    save_plots=True,
    save_model=True
)

print("Pipeline konfigürasyonu oluşturuldu")
print(f"\nÖnemli ayarlar:")
print(f"  Seçim sırası: {config.selection_order}")
print(f"  Seçim yöntemi: {config.selection_method}")
print(f"  Binning yöntemi: {config.binning_method}")
print(f"  Eşit default oranları: {config.equal_default_splits}")
print(f"  Kalibrasyon aktif: {config.enable_calibration}")
print(f"  Model tipi: {config.model_type}")

Pipeline konfigürasyonu oluşturuldu

Önemli ayarlar:
  Seçim sırası: ['psi', 'vif', 'correlation', 'iv', 'boruta', 'stepwise']
  Seçim yöntemi: stepwise
  Binning yöntemi: optimized
  Eşit default oranları: True
  Kalibrasyon aktif: True
  Model tipi: all


## 5. Pipeline'ı Çalıştır

In [6]:
# Unified pipeline'ı oluştur
pipeline = UnifiedRiskPipeline(config=config)

print("Pipeline başlatıldı!")
print(f"\nBileşenler:")
print(f"  Data Processor: {type(pipeline.data_processor).__name__}")
print(f"  WOE Transformer: {type(pipeline.woe_transformer).__name__}")
print(f"  Feature Selector: {type(pipeline.feature_selector).__name__}")
print(f"  Model Builder: {type(pipeline.model_builder).__name__}")
print(f"  Calibrator: {type(pipeline.calibrator).__name__}")
print(f"  Risk Band Optimizer: {type(pipeline.risk_band_optimizer).__name__}")
print(f"  Reporter: {type(pipeline.reporter).__name__}")

Pipeline başlatıldı!

Bileşenler:
  Data Processor: DataProcessor
  WOE Transformer: EnhancedWOETransformer
  Feature Selector: AdvancedFeatureSelector
  Model Builder: ComprehensiveModelBuilder
  Calibrator: TwoStageCalibrator
  Risk Band Optimizer: OptimalRiskBandAnalyzer
  Reporter: EnhancedReporter


In [7]:
# Pipeline'ı çalıştır
print("Pipeline çalıştırılıyor...")
print("="*80)

results = pipeline.fit(
    df=data,
    data_dictionary=data_dictionary,
    calibration_df=calibration_data,
    stage2_df=stage2_data
)

print("\n" + "="*80)
print("Pipeline başarıyla tamamlandı!")

Pipeline çalıştırılıyor...
UNIFIED RISK PIPELINE EXECUTION

[Step 1/10] Data Processing...
  Found 8 numeric and 6 categorical features

[Step 2/10] Data Splitting...
    Performing equal default rate split...
      [OK] Equal default rates achieved (mean: 12.82%, std: 0.0004)
  oot: 729 samples, default rate: 12.76%
  train: 11416 samples, default rate: 12.85%
  test: 2855 samples, default rate: 12.85%

[Step 3/10] WOE Transformation & Univariate Analysis...
  Processing age...
  Processing income...
  Processing loan_amount...
  Processing employment_years...
  Processing credit_score...
  Processing debt_to_income...
  Processing num_credit_lines...
  Processing months_since_delinquent...
  Processing total_credit_limit...
  Processing utilization_rate...
  Processing home_ownership...
  Processing loan_purpose...
  Processing employment_type...
  Processing education...
  Processing marital_status...
  Processing state...
  Processing snapshot_month...
  Processing noise_sentinel..

## 6. Sonuçları Analiz Et

In [ ]:
# Model performansları
print("MODEL PERFORMANS ÖZETİ")
print("="*60)

if 'model_results' in results:
    model_scores = results['model_results'].get('scores', {})
    if model_scores:
        scores_df = pd.DataFrame(model_scores).T
        print(scores_df.round(4))
        
        # En iyi model
        best_model = results['model_results'].get('best_model_name', 'N/A')
        print(f"\nEn İyi Model: {best_model}")

# Seçilen özellikler
if 'selection_results' in results:
    features = results['selection_results'].get('selected_features', [])
    print(f"\nSeçilen Özellik Sayısı: {len(features)}")
    print(f"İlk 10 özellik: {features[:10]}")

In [ ]:
# WOE ve Univariate Gini analizi
print("WOE VE UNIVARIATE GINI ANALİZİ")
print("="*60)

if 'woe_results' in results:
    woe_res = results['woe_results']
    
    if 'univariate_gini' in woe_res:
        gini_data = []
        for var, gini_info in woe_res['univariate_gini'].items():
            gini_data.append({
                'Variable': var,
                'Gini (Raw)': gini_info.get('gini_raw', 0),
                'Gini (WOE)': gini_info.get('gini_woe', 0),
                'Gini Drop': gini_info.get('gini_drop', 0)
            })
        
        gini_df = pd.DataFrame(gini_data)
        gini_df = gini_df.sort_values('Gini (WOE)', ascending=False)
        print(gini_df.head(10).round(4))
        
        # WOE kalitesi kontrolü
        problematic = gini_df[gini_df['Gini Drop'] > 0.05]
        if len(problematic) > 0:
            print(f"\n⚠ WOE sonrası Gini düşüşü olan değişkenler: {len(problematic)}")

In [ ]:
# Risk band analizi
print("RISK BAND ANALİZİ")
print("="*60)

if 'risk_bands' in results:
    rb = results['risk_bands']
    
    if 'bands' in rb:
        bands_df = rb['bands']
        print(bands_df[['band', 'count', 'bad_rate', 'avg_score', 'ks']].round(4))
    
    if 'metrics' in rb:
        metrics = rb['metrics']
        print(f"\nRisk Band Metrikleri:")
        print(f"  Herfindahl Index: {metrics.get('herfindahl_index', 0):.4f}")
        print(f"  Entropy: {metrics.get('entropy', 0):.4f}")
        print(f"  Gini Coefficient: {metrics.get('gini_coefficient', 0):.4f}")
        print(f"  Hosmer-Lemeshow p-value: {metrics.get('hosmer_lemeshow_p', 0):.4f}")
        print(f"  KS Statistic: {metrics.get('ks_stat', 0):.4f}")
        
        # Binomial test sonuçları
        if 'binomial_tests' in metrics:
            significant = sum(1 for b in metrics['binomial_tests'].values() 
                            if b.get('significant', False))
            print(f"\nBinomial Test Sonuçları:")
            print(f"  Anlamlı band sayısı: {significant}/{len(metrics['binomial_tests'])}")

In [ ]:
# Kalibrasyon analizi
print("KALİBRASYON ANALİZİ")
print("="*60)

if 'calibration_stage1' in results:
    stage1 = results['calibration_stage1']
    if 'calibration_metrics' in stage1:
        s1_metrics = stage1['calibration_metrics']
        print("Stage 1 Kalibrasyon (Uzun dönem):")
        print(f"  ECE: {s1_metrics.get('ece', 0):.4f}")
        print(f"  MCE: {s1_metrics.get('mce', 0):.4f}")
        print(f"  Brier Score: {s1_metrics.get('brier', 0):.4f}")
        print(f"  Calibration Gap: {s1_metrics.get('calibration_gap', 0):.4f}")

if 'calibration_stage2' in results:
    stage2 = results['calibration_stage2']
    if 'stage2_metrics' in stage2:
        s2_metrics = stage2['stage2_metrics']
        print("\nStage 2 Kalibrasyon (Son dönem):")
        print(f"  ECE: {s2_metrics.get('ece', 0):.4f}")
        print(f"  MCE: {s2_metrics.get('mce', 0):.4f}")
        print(f"  Mean Predicted: {s2_metrics.get('mean_predicted', 0):.4f}")
        print(f"  Mean Actual: {s2_metrics.get('mean_actual', 0):.4f}")

## 7. Görselleştirmeler

In [ ]:
# Görselleştirmeler
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Model AUC karşılaştırması
if 'model_results' in results and 'scores' in results['model_results']:
    ax = axes[0, 0]
    scores = results['model_results']['scores']
    models = list(scores.keys())
    train_aucs = [scores[m].get('train_auc', 0) for m in models]
    test_aucs = [scores[m].get('test_auc', 0) for m in models]
    
    x = np.arange(len(models))
    width = 0.35
    
    ax.bar(x - width/2, train_aucs, width, label='Train', color='steelblue')
    ax.bar(x + width/2, test_aucs, width, label='Test', color='orange')
    ax.set_xlabel('Model')
    ax.set_ylabel('AUC')
    ax.set_title('Model Performans Karşılaştırması')
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=45, ha='right')
    ax.legend()
    ax.grid(True, alpha=0.3)

# 2. Risk Band dağılımı
if 'risk_bands' in results and 'bands' in results['risk_bands']:
    ax = axes[0, 1]
    bands_df = results['risk_bands']['bands']
    ax.bar(bands_df['band'], bands_df['count'], color='coral')
    ax.set_xlabel('Risk Band')
    ax.set_ylabel('Kayıt Sayısı')
    ax.set_title('Risk Band Dağılımı')
    ax.grid(True, alpha=0.3)

# 3. Default oranı by Risk Band
if 'risk_bands' in results and 'bands' in results['risk_bands']:
    ax = axes[0, 2]
    bands_df = results['risk_bands']['bands']
    ax.plot(bands_df['band'], bands_df['bad_rate'], 'o-', color='darkred', linewidth=2)
    ax.set_xlabel('Risk Band')
    ax.set_ylabel('Default Oranı')
    ax.set_title('Risk Band Default Oranları')
    ax.grid(True, alpha=0.3)

# 4. IV değerleri
if 'woe_results' in results and 'woe_values' in results['woe_results']:
    ax = axes[1, 0]
    woe_values = results['woe_results']['woe_values']
    iv_data = [(var, info.get('iv', 0)) for var, info in woe_values.items()]
    iv_data = sorted(iv_data, key=lambda x: x[1], reverse=True)[:10]
    
    vars_top = [x[0] for x in iv_data]
    ivs_top = [x[1] for x in iv_data]
    
    ax.barh(range(len(vars_top)), ivs_top, color='green')
    ax.set_yticks(range(len(vars_top)))
    ax.set_yticklabels(vars_top)
    ax.set_xlabel('Information Value')
    ax.set_title('Top 10 Değişken (IV)')
    ax.invert_yaxis()

# 5. KS eğrisi
if 'risk_bands' in results and 'bands' in results['risk_bands']:
    ax = axes[1, 1]
    bands_df = results['risk_bands']['bands']
    ax.plot(bands_df['cum_count'] / bands_df['cum_count'].max(),
           bands_df['bad_capture'], 'b-', label='Bad Capture', linewidth=2)
    ax.plot(bands_df['cum_count'] / bands_df['cum_count'].max(),
           bands_df['good_capture'], 'g-', label='Good Capture', linewidth=2)
    ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
    ax.set_xlabel('Popülasyon %')
    ax.set_ylabel('Kümülatif %')
    ax.set_title('KS Eğrisi')
    ax.legend()
    ax.grid(True, alpha=0.3)

# 6. Kalibrasyon grafiği
ax = axes[1, 2]
ax.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Perfect Calibration')
# Gerçek kalibrasyon verisi varsa eklenebilir
ax.set_xlabel('Tahmin Edilen Olasılık')
ax.set_ylabel('Gerçekleşen Oran')
ax.set_title('Kalibrasyon Grafiği')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Deployment Kodu Üretimi

In [ ]:
# SQL ve Python deployment kodları
print("DEPLOYMENT KODU ÜRETİMİ")
print("="*60)

if hasattr(pipeline.risk_band_optimizer, 'export_sql'):
    if 'risk_bands' in results and 'bands' in results['risk_bands']:
        sql_code = pipeline.risk_band_optimizer.export_sql(results['risk_bands']['bands'])
        print("SQL Kodu (Risk Bands):")
        print("-"*40)
        print(sql_code[:500])
        if len(sql_code) > 500:
            print("...\n[Kısaltıldı]")

if hasattr(pipeline.risk_band_optimizer, 'export_python'):
    if 'risk_bands' in results and 'bands' in results['risk_bands']:
        python_code = pipeline.risk_band_optimizer.export_python(results['risk_bands']['bands'])
        print("\nPython Kodu (Risk Bands):")
        print("-"*40)
        print(python_code[:500])
        if len(python_code) > 500:
            print("...\n[Kısaltıldı]")

## 9. Pipeline'ı Kaydet

In [ ]:
# Pipeline'ı kaydet
import os
import joblib

output_dir = '../outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model_path = os.path.join(output_dir, 'unified_pipeline.pkl')
pipeline.save_pipeline(model_path)

print(f"Pipeline kaydedildi: {model_path}")

# Sonuçları da kaydet
results_path = os.path.join(output_dir, 'pipeline_results.pkl')
joblib.dump(results, results_path)
print(f"Sonuçlar kaydedildi: {results_path}")

## 10. Final Özet

In [ ]:
# Final özet raporu
print("="*80)
print("UNIFIED RISK MODEL PIPELINE - FİNAL ÖZET")
print("="*80)
print()

summary = []

# Veri özeti
summary.append("VERİ ÖZETİ:")
summary.append(f"  Toplam kayıt: {len(data)}")
summary.append(f"  Özellik sayısı: {len(data.columns) - 3}")
summary.append(f"  Default oranı: {data['default'].mean():.2%}")
summary.append("")

# Pipeline konfigürasyonu
summary.append("PIPELINE KONFİGÜRASYONU:")
summary.append(f"  Seçim yöntemi: {config.selection_method}")
summary.append(f"  Binning yöntemi: {config.binning_method}")
summary.append(f"  Kalibrasyon: Stage 1 ({config.calibration_method}) + Stage 2 ({config.stage2_method})")
summary.append(f"  Risk bands: {config.n_risk_bands} band, {config.band_method} yöntemi")
summary.append("")

# Model performansı
if 'model_results' in results:
    summary.append("MODEL PERFORMANSI:")
    best_model = results['model_results'].get('best_model_name', 'N/A')
    summary.append(f"  En iyi model: {best_model}")
    
    if 'scores' in results['model_results'] and best_model in results['model_results']['scores']:
        best_scores = results['model_results']['scores'][best_model]
        summary.append(f"  Train AUC: {best_scores.get('train_auc', 0):.4f}")
        summary.append(f"  Test AUC: {best_scores.get('test_auc', 0):.4f}")
    summary.append("")

# Risk band metrikleri
if 'risk_bands' in results and 'metrics' in results['risk_bands']:
    metrics = results['risk_bands']['metrics']
    summary.append("RISK BAND METRİKLERİ:")
    summary.append(f"  Herfindahl Index: {metrics.get('herfindahl_index', 0):.4f}")
    summary.append(f"  KS Statistic: {metrics.get('ks_stat', 0):.4f}")
    summary.append(f"  Hosmer-Lemeshow p: {metrics.get('hosmer_lemeshow_p', 0):.4f}")
    summary.append("")

# Test edilen özellikler
summary.append("TEST EDİLEN ÖZELLİKLER:")
summary.append("  ✓ Tek unified pipeline ile tüm kontrol")
summary.append("  ✓ WOE transformasyonu ve univariate Gini")
summary.append("  ✓ PSI -> VIF -> Correlation -> IV -> Boruta -> Stepwise seçim")
summary.append("  ✓ Tüm ML modelleri (Logistic, GAM, CatBoost, LightGBM, XGBoost, RF, ExtraTrees)")
summary.append("  ✓ Stage 1 ve Stage 2 kalibrasyon")
summary.append("  ✓ Risk band optimizasyonu (Herfindahl, Hosmer-Lemeshow, Binomial)")
summary.append("  ✓ Eşit default oranları ile veri bölme")
summary.append("  ✓ Data dictionary entegrasyonu")
summary.append("  ✓ Deployment kodu üretimi")

# Özeti yazdır
for line in summary:
    print(line)

print("\n" + "="*80)
print("Pipeline testi başarıyla tamamlandı!")
print("="*80)